# Yggdrax Getting Started\n\nThis notebook demonstrates basic tree construction, geometry extraction, and interaction generation.

In [ ]:
import jax\nimport jax.numpy as jnp\n\nfrom yggdrax import build_tree, compute_tree_geometry, build_interactions_and_neighbors

In [ ]:
key = jax.random.PRNGKey(7)\nkey_pos, key_mass = jax.random.split(key)\npositions = jax.random.uniform(key_pos, (256, 3), minval=-1.0, maxval=1.0, dtype=jnp.float32)\nmasses = jax.random.uniform(key_mass, (256,), minval=0.5, maxval=1.5, dtype=jnp.float32)

In [ ]:
tree = build_tree(positions, masses, leaf_size=16)\npositions_sorted = positions[tree.sorted_indices]\ngeometry = compute_tree_geometry(tree, positions_sorted)\ninteractions, neighbors = build_interactions_and_neighbors(tree, geometry)\n\nprint('nodes:', int(tree.num_nodes))\nprint('leaves:', int(tree.num_leaves))\nprint('far-field pairs:', int(interactions.count))\nprint('near-field pairs:', int(neighbors.count))